# For figure 6.25

Temperature response to forcing 2020-2100

In [1]:
import numpy as np
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as pl
import os
from matplotlib import gridspec, rc
from matplotlib.lines import Line2D
import matplotlib.patches as mp
from netCDF4 import Dataset
import warnings

from ar6.utils.h5 import *

In [2]:
# TODO - sort out figure sizing

pl.rcParams['figure.figsize'] = (18/2.54, 11/2.54)
pl.rcParams['font.size'] = 11
pl.rcParams['font.family'] = 'Arial'
pl.rcParams['xtick.direction'] = 'out'
pl.rcParams['xtick.minor.visible'] = True
pl.rcParams['ytick.major.left'] = True
pl.rcParams['ytick.major.size'] = 0
pl.rcParams['xtick.top'] = True
pl.rcParams['figure.dpi'] = 150

In [3]:
results = load_dict_from_hdf5('../data_output_large/twolayer_SSPs.h5')

In [4]:
results.keys()

dict_keys(['ssp119', 'ssp119_climuncert', 'ssp119_remove_aerosol', 'ssp119_remove_aerosol_climuncert', 'ssp119_remove_bc_on_snow', 'ssp119_remove_bc_on_snow_climuncert', 'ssp119_remove_ch4', 'ssp119_remove_ch4_climuncert', 'ssp119_remove_co2', 'ssp119_remove_co2_climuncert', 'ssp119_remove_hfc', 'ssp119_remove_hfc_climuncert', 'ssp119_remove_o3', 'ssp119_remove_o3_climuncert', 'ssp119_remove_other_anthro', 'ssp119_remove_other_anthro_climuncert', 'ssp119_remove_other_ghg', 'ssp119_remove_other_ghg_climuncert', 'ssp119_remove_slcf', 'ssp119_remove_slcf_climuncert', 'ssp126', 'ssp126_climuncert', 'ssp126_remove_aerosol', 'ssp126_remove_aerosol_climuncert', 'ssp126_remove_bc_on_snow', 'ssp126_remove_bc_on_snow_climuncert', 'ssp126_remove_ch4', 'ssp126_remove_ch4_climuncert', 'ssp126_remove_co2', 'ssp126_remove_co2_climuncert', 'ssp126_remove_hfc', 'ssp126_remove_hfc_climuncert', 'ssp126_remove_o3', 'ssp126_remove_o3_climuncert', 'ssp126_remove_other_anthro', 'ssp126_remove_other_anthro_cl

In [5]:
results['ssp245']['surface_temperature'].shape

(351, 2565)

In [6]:
results['ssp245']['surface_temperature'][269].mean()

1.1905754111653173

In [8]:
forcings = ['ch4', 'hfc', 'o3', 'aerosol', 'bc_on_snow', 'slcf']
scenarios = ['ssp119','ssp126','ssp245','ssp370','ssp370-lowNTCF-aerchemmip','ssp370-lowNTCF-gidden','ssp434','ssp460','ssp534-over','ssp585']
for scenario in scenarios:
    for forcing in forcings:
        print(scenario, forcing, (results[scenario]['surface_temperature'][331:351] - results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][331:351]).mean())

ssp119 ch4 0.14987559127594505
ssp119 hfc 0.012803136637740365
ssp119 o3 0.1060682539059846
ssp119 aerosol -0.18392138177359327
ssp119 bc_on_snow 0.007354868209108782
ssp119 slcf 0.09218046825515318
ssp126 ch4 0.15882631981555062
ssp126 hfc 0.01548947806292778
ssp126 o3 0.11098729604578636
ssp126 aerosol -0.18733279973961608
ssp126 bc_on_snow 0.010647044053500488
ssp126 slcf 0.10861733823901872
ssp245 ch4 0.2844913480358258
ssp245 hfc 0.06304149777135094
ssp245 o3 0.2028181583177023
ssp245 aerosol -0.317866269964508
ssp245 bc_on_snow 0.018186686585097568
ssp245 slcf 0.25067142074529464
ssp370 ch4 0.4934896503400651
ssp370 hfc 0.09034587712447495
ssp370 o3 0.41901370378453934
ssp370 aerosol -0.6624943077303214
ssp370 bc_on_snow 0.06680516715441735
ssp370 slcf 0.40716009067322484
ssp370-lowNTCF-aerchemmip ch4 0.4934896503400207
ssp370-lowNTCF-aerchemmip hfc 0.0903458771245214
ssp370-lowNTCF-aerchemmip o3 0.3504438512777137
ssp370-lowNTCF-aerchemmip aerosol -0.3288867553188828
ssp370-lowN

In [9]:
(results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][331:351] - results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][0]).shape

(20, 2565)

In [19]:
forcings = ['ch4', 'hfc', 'o3', 'aerosol', 'bc_on_snow', 'slcf']

AR6_forc = {}
AR6_ecsforc = {}

base_periods = {
#    '1750': 0,
#    '1850-1900': slice(100,151),
#    '1995-2014': slice(245,265),
#    '2010-2019': slice(260,270),
    '2020'     : slice(270,271)
}

for scenario in scenarios:
    AR6_forc[scenario] = {}
    AR6_ecsforc[scenario] = {}
    for forcing in forcings:
        AR6_forc[scenario][forcing] = {}
        AR6_ecsforc[scenario][forcing] = {}
        for base_period in base_periods:
            AR6_forc[scenario][forcing][base_period] = np.zeros((81,5))
            AR6_ecsforc[scenario][forcing][base_period] = np.zeros((81,5))
            for year in range(270,351):
                AR6_forc[scenario][forcing][base_period][year-270,:] = np.percentile(
                    (
                        (results[scenario]['surface_temperature'][year]- results[scenario]['surface_temperature'][base_periods[base_period]].mean(axis=0))-
                        (results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][year] - results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][base_periods[base_period]].mean(axis=0))
                    ), (5,16,50,84,95)
                )
                AR6_ecsforc[scenario][forcing][base_period][year-270,:] = np.percentile(
                    (
                        (results['%s_climuncert' % scenario]['surface_temperature'][year] - results['%s_climuncert' % scenario]['surface_temperature'][base_periods[base_period]].mean(axis=0))-
                        (results['%s_remove_%s_climuncert' % (scenario, forcing)]['surface_temperature'][year] - results['%s_remove_%s_climuncert' % (scenario, forcing)]['surface_temperature'][base_periods[base_period]].mean(axis=0))
                    ), (5,16,50,84,95)
                )
    AR6_forc[scenario]['anthro'] = {}
    AR6_ecsforc[scenario]['anthro'] = {}
    for base_period in base_periods:
        AR6_forc[scenario]['anthro'][base_period] = np.zeros((81,5))
        AR6_ecsforc[scenario]['anthro'][base_period] = np.zeros((81,5))
        for year in range(270,351):
            AR6_forc[scenario]['anthro'][base_period][year-270,:] = np.percentile(results[scenario]['surface_temperature'][year] - results[scenario]['surface_temperature'][base_periods[base_period]].mean(axis=0), (5,16,50,84,95))
            AR6_ecsforc[scenario]['anthro'][base_period][year-270,:] = np.percentile(results['%s_climuncert' % scenario]['surface_temperature'][year] - results['%s_climuncert' % scenario]['surface_temperature'][base_periods[base_period]].mean(axis=0), (5,16,50,84,95))

In [20]:
print(AR6_ecsforc['ssp245']['anthro']['2020'])

[[0.         0.         0.         0.         0.        ]
 [0.0197922  0.02255309 0.02865883 0.0357961  0.04120079]
 [0.03926918 0.04448751 0.05634829 0.07023835 0.08062842]
 [0.05834673 0.0659204  0.08322847 0.10382305 0.11834807]
 [0.07651593 0.08673876 0.1097026  0.136556   0.1563727 ]
 [0.09466204 0.10719497 0.1354256  0.16831622 0.1929167 ]
 [0.11283321 0.12748225 0.16054467 0.20030709 0.2288654 ]
 [0.13014089 0.14783785 0.185027   0.23127856 0.26487452]
 [0.14755288 0.16795783 0.20997216 0.26241023 0.29962488]
 [0.16468416 0.18766274 0.23451374 0.2932809  0.33492901]
 [0.18173777 0.20710886 0.25891341 0.32397001 0.36928431]
 [0.19934637 0.22691266 0.28304381 0.35523487 0.40449891]
 [0.2171592  0.24677466 0.30788922 0.38684637 0.43967965]
 [0.2351787  0.26731539 0.33340513 0.41817287 0.47529928]
 [0.25315027 0.28786604 0.35865277 0.45030361 0.5121339 ]
 [0.27088522 0.30836974 0.38405742 0.48198889 0.54993588]
 [0.28839413 0.32893279 0.40896494 0.51453515 0.58702757]
 [0.30588097 0

In [22]:
for scenario in scenarios:
    print(scenario, AR6_ecsforc[scenario]['anthro']['2020'][:,[0,2,4]])

ssp119 [[ 0.          0.          0.        ]
 [ 0.01986871  0.0314212   0.04616757]
 [ 0.03885297  0.0627449   0.09415625]
 [ 0.05665372  0.0935242   0.14239246]
 [ 0.0727525   0.12338227  0.1903041 ]
 [ 0.0880317   0.15199308  0.23786549]
 [ 0.10186225  0.17928006  0.28530547]
 [ 0.11438918  0.20585742  0.33114564]
 [ 0.12583617  0.23039158  0.37589513]
 [ 0.13504377  0.25374718  0.41947428]
 [ 0.1434561   0.27618144  0.46372792]
 [ 0.15006499  0.29542179  0.50304692]
 [ 0.15478174  0.31066188  0.53461466]
 [ 0.157996    0.32294589  0.56230679]
 [ 0.15955386  0.33281027  0.58466309]
 [ 0.16024564  0.34014382  0.60451374]
 [ 0.16039486  0.34606927  0.6209168 ]
 [ 0.15980059  0.35043082  0.63597119]
 [ 0.15859997  0.35400831  0.64872555]
 [ 0.15707843  0.35649842  0.65750563]
 [ 0.15526246  0.35744377  0.6659641 ]
 [ 0.15298857  0.35852002  0.67332188]
 [ 0.15026183  0.35837671  0.67816429]
 [ 0.14824789  0.35824809  0.68240935]
 [ 0.14550947  0.35679261  0.68698261]
 [ 0.14283334  0.3

In [25]:
output = []
for scenario in scenarios:
    for forcing in ['ch4', 'hfc', 'o3', 'aerosol', 'bc_on_snow', 'slcf', 'anthro']:
        for year in range(270, 351):
            output.append([scenario, forcing, '2020', year+1750,
               AR6_ecsforc[scenario][forcing]['2020'][year-270,0],
               AR6_ecsforc[scenario][forcing]['2020'][year-270,1],
               AR6_ecsforc[scenario][forcing]['2020'][year-270,2],
               AR6_ecsforc[scenario][forcing]['2020'][year-270,3],
               AR6_ecsforc[scenario][forcing]['2020'][year-270,4],
            ])

df = pd.DataFrame(output, columns=['scenario', 'forcing', 'base_period', 'year', 'p05','p16','p50','p84','p95'])
df.to_csv('../data_output/slcf_warming_ranges.csv')
df

,scenario,forcing,base_period,year,p05,p16,p50,p84,p95
0,ssp119,ch4,2020,2020,0.000000,0.000000,0.000000,0.000000,0.000000
1,ssp119,ch4,2020,2021,0.001217,0.001424,0.001799,0.002359,0.002796
2,ssp119,ch4,2020,2022,0.002162,0.002547,0.003276,0.004365,0.005238
3,ssp119,ch4,2020,2023,0.002799,0.003353,0.004393,0.005970,0.007246
4,ssp119,ch4,2020,2024,0.003127,0.003815,0.005125,0.007147,0.008850
...,...,...,...,...,...,...,...,...,...
5665,ssp585,anthro,2020,2096,2.414035,2.745542,3.375208,4.241771,4.928117
5666,ssp585,anthro,2020,2097,2.444173,2.780960,3.419501,4.299079,4.995392
5667,ssp585,anthro,2020,2098,2.475532,2.816396,3.463092,4.357588,5.062269
5668,ssp585,anthro,2020,2099,2.506110,2.852463,3.506855,4.417798,5.129299
